#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**

In [263]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=123
)

In [264]:
a = np.array([1, 2, 3, 1, 1])
c = Counter(a)
c.most_common(1)[0][0]

1

In [265]:
def entropy_func(class_count, num_samples):
    probability = class_count / num_samples
    return -(probability) * math.log(probability, 2)


class Group:
    def __init__(self, group_classes):
        self.group_classes = group_classes
        self.entropy = self.group_entropy()

    def __len__(self):
        return self.group_classes.size

    # assuming that group_classes is an array of classes
    def group_entropy(self):
        # determine classes of data ponits
        _, counts = np.unique(self.group_classes, return_counts=True)
        entropy = np.sum([entropy_func(count, len(self)) for count in counts])
        return entropy

In [266]:
class Node:
    def __init__(
        self,
        split_feature,
        split_val,
        depth=None,
        child_node_a=None,
        child_node_b=None,
        val=None,
    ):
        self.split_feature = split_feature
        self.split_val = split_val
        self.depth = depth
        self.child_node_a = child_node_a
        self.child_node_b = child_node_b
        self.val = val

    def predict(self, data):
        if not self.val == None:
            return self.val
        elif data[self.split_feature] < self.split_val:
            return self.child_node_a.predict(data)
        else:
            return self.child_node_b.predict(data)

In [267]:
a = np.array([1, 2, 3, 2, 1, 3])
b = np.array([i for i in range(len(a)) if a[i] < 3])
c = np.array([i for i in range(len(a)) if not i in b])


print(c)

[2 5]


In [268]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        self.tree = None

    @staticmethod
    def get_split_entropy(group_a, group_b):
        return (group_a.entropy * len(group_a) + group_b.entropy * len(group_b)) / (
            len(group_a) + len(group_b)
        )

    def get_information_gain(self, parent_group, child_group_a, child_group_b):
        return parent_group.entropy - self.get_split_entropy(
            child_group_a, child_group_b
        )

    def get_best_feature_split(self, feature_values, classes):
        best_information_gain = 0
        best_feature_split = None
        for value in feature_values:
            indices_meeting_condition = [
                i for i in range(len(classes)) if feature_values[i] < value
            ]
            indices_not_meeting_condition = [
                i for i in range(len(classes)) if not i in indices_meeting_condition
            ]
            informtion_gain = self.get_information_gain(
                Group(classes),
                Group(classes[indices_meeting_condition]),
                Group(classes[indices_not_meeting_condition]),
            )
            if informtion_gain > best_information_gain:
                best_information_gain = informtion_gain
                best_feature_split = value
        return best_information_gain, best_feature_split

    def get_best_split(self, data, classes):
        best_information_gain = 0
        best_feature = None
        best_split = None
        for i in range(len(data[0])):
            information_gain, split_value = self.get_best_feature_split(
                data[:, i], classes
            )
            if information_gain > best_information_gain:
                best_information_gain = information_gain
                best_feature = i
                best_split = split_value
        return best_feature, best_split

    def build_tree(self, data, classes, depth=0):
        if len(np.unique(classes)) == 1:
            return Node(None, None, val = classes[0])
        if all(all(element == data[0]) for element in data):
            return Node(None, None, val = Counter(classes).most_common()[0][0])
        if depth == self.max_depth:
            return Node(None, None, val = Counter(classes).most_common()[0][0])
        best_feature, best_split = self.get_best_split(data, classes)
        indices_meeting_condition = [
            i for i in range(len(data)) if data[i][best_feature] < best_split
        ]
        indices_not_meeting_condition = [
            i for i in range(len(data)) if not i in indices_meeting_condition
        ]
        child_a_value = None
        child_b_value = None
        if len(indices_meeting_condition) == 0:
            child_a_value = Counter(classes).most_common()[0][0]
        if len(indices_not_meeting_condition) == 0:
            child_b_value = Counter(classes).most_common()[0][0]
        if not child_a_value:
            child_node_a = self.build_tree(data[indices_meeting_condition], classes[indices_meeting_condition], depth + 1)
        else:
            child_node_a = Node(None, None, val = child_a_value)
        if not child_b_value:
            child_node_b = self.build_tree(data[indices_not_meeting_condition], classes[indices_not_meeting_condition], depth + 1)
        else:
            child_node_b = Node(None, None, val= child_b_value)
        return Node(best_feature, best_split, depth, child_node_a, child_node_b)
        
        
    def predict(self, data):
        return self.tree.predict(data)

In [269]:
dc = DecisionTreeClassifier(100)
dc.tree = dc.build_tree(x_train, y_train)

predictions = []
for sample, gt in zip(x_test, y_test):
    prediction = dc.predict(sample)
    predictions.append(prediction)
print(sum(predictions == y_test)/len(y_test))


0.9333333333333333


In [270]:
print(all(all(element == d[0]) for element in d))

True
